In [8]:
import pandas as pd
import numpy as np
import polars as pl


In [9]:
folder = '.\\input_files\\'
file = '941279779_adventist-health---st.-helena_standardcharges.csv'

dtypes = {
    'Procedure Code': str, 
    'Revenue Code': str, 
    'Modifier1': str,
    'Modifier2': str,   
    'Rx Unit Multiplier': str 
}
df = pd.read_csv(folder + file, dtype=dtypes, skiprows=3)


C:\Users\adria\AppData\Local\Temp\ipykernel_21212\3491677796.py:11: DtypeWarning: Columns (7,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(folder + file, dtype=dtypes, skiprows=3)


In [10]:
df.rename(columns={
    'Procedure Code': 'local_code',
    'Procedure Description': 'description',
    'Price Tier': 'setting',
    'Revenue Code': 'rev_code',
    'CPT HCPCS Code': 'code',
    'NDC Code': 'ndc',
    'Rx Unit Multiplier': 'drug_hcpcs_multiplier'
    
}, inplace=True)

In [11]:
df = df[~df['local_code'].isin(['Diagnosis Related Group Code', 'Shoppable Services Code'])]

In [12]:
cols = df.columns.tolist()
id_vars = cols[:9]
value_vars = cols[9:]


df = pd.melt(df, id_vars=id_vars, value_vars=value_vars, var_name='payer', value_name='rate')

In [13]:
df['ndc'].fillna(pd.NA, inplace=True)

In [14]:
mapping = {
    'Gross Charge': 'gross',
    'Discounted Cash Price': 'cash',
    'De-identified minimum negotiated charge': 'min',
    'De-identified maximum negotiated charge': 'max'
}

df['payer_category'] = df['payer'].map(mapping).fillna('payer')

In [15]:
df['plan'] = ''
df['plan'] = np.where(df['payer'].str.contains('ALL_PAYER_-_ALL_OTHER_PLANS'), 'ALL PAYER - ALL OTHER PLANS',
                      np.where(df['payer'].str.contains('ALL_OTHER_PLANS'), 'ALL OTHER PLANS',
                               np.where(df['payer'].str.contains('ALL_PLANS'), 'ALL PLANS', df['plan'])))


df['payer'] = df['payer'].str.replace('ALL_PAYER_-_ALL_OTHER_PLANS|ALL_PLANS|ALL_OTHER_PLANS', '', regex=True).str.strip('-_')

In [16]:
df.loc[df['drug_hcpcs_multiplier'] == '0', 'drug_hcpcs_multiplier'] = pd.NA

In [17]:
df['modifiers'] = df['Modifier1'] + df['Modifier2']
df.drop(['Modifier1', 'Modifier2'], axis=1, inplace=True)

In [18]:
cols = ['setting', 'code', 'ndc', 'local_code', 'plan', 'modifiers']
df[cols] = df[cols].astype(str)
df.loc[:, cols].fillna('', inplace=True)

In [19]:
df['setting'].fillna('', inplace=True)

In [20]:
df['rate'] = df['rate'].astype(float)

In [21]:
df.loc[df['code'].str.match(r'^\d{3}$'), 'ms_drg'] = df['code']

In [22]:
df.loc[df['ndc'].isin(['<NA>', 'nan']), 'ndc'] = ''
df.loc[df['setting'].isin(['<NA>', 'nan']),'setting'] = ''
df.loc[df['modifiers'].isin(['<NA>', 'nan']),'modifiers'] = ''

df['ms_drg'].fillna('', inplace=True)
df['rev_code'].fillna('', inplace=True)

df['drug_hcpcs_multiplier'].fillna(pd.NA, inplace=True)

In [23]:
df.loc[df['local_code'] != df['code'], 'temp_cpt'] = df['code']

In [24]:
df.loc[(df['local_code'] != df['code']) & (df['code'].str.len() == 5), 'hcpcs_cpt'] = df['code']
df.loc[(df['local_code'] != df['code']) & (df['code'].str.len() == 3), 'ms_drg'] = df['code']
df.loc[(df['local_code'] == df['code']) & (df['code'].str.len() != 5) & (df['code'].str.len() != 3), 'icd'] = df['code']

In [26]:
setting_map = {
        'ambulatory surgical': 'outpatient',
        'emergency': 'emergency',
        'observation': 'outpatient',
        'urgent care': 'outpatient',
        'profee': 'inpatient',
        'inpatient': 'inpatient',
        'outpatient': 'outpatient'
    }

df.loc[df['setting'].isin(setting_map.keys()), 'additional_payer_notes'] = f"Original patient setting: {df['setting']}"

df['setting'] = df['setting'].map(setting_map).fillna('1')

In [27]:
df3 = df

In [29]:
if file == '812240617_adventist-health---tehachapi-valley_standardcharges.csv':
    df['hospital_id'] = '051301'
elif file == '812240617_adventist-health-ukiah-valley_standardcharges.csv':
    df['hospital_id'] = '051302'
elif file == '941279779_adventist-health---st.-helena_standardcharges.csv':
    print("a")
    df1 = df.copy()
    df1['hospital_id'] = '054074'
    df['hospital_id'] = '050013'
    df = pd.concat([df, df1])

a


In [47]:
df = df3

In [48]:
df1 = df.copy()
df1['hospital_id'] = '054074'
df['hospital_id'] = '050013'

In [49]:
df2 = pd.concat([df, df1])

In [50]:
df2

,local_code,description,setting,rev_code,code,ndc,drug_hcpcs_multiplier,payer,rate,payer_category,plan,modifiers,ms_drg,temp_cpt,hcpcs_cpt,icd,additional_payer_notes,hospital_id
0,1000006_1,Culture Wound wo Gram Stain,1,0306,87070,,NaN,Gross Charge,490.00,gross,,,,87070,87070,NaN,NaN,050013
1,1000006_1,Culture Wound wo Gram Stain,1,0306,87070,,NaN,Gross Charge,490.00,gross,,,,87070,87070,NaN,NaN,050013
2,1000181_1,Endovasc DTA WO LT S&I,1,0320,75957,,NaN,Gross Charge,2064.00,gross,,,,75957,75957,NaN,NaN,050013
3,1000181_1,Endovasc DTA WO LT S&I,1,0320,75957,,NaN,Gross Charge,2064.00,gross,,,,75957,75957,NaN,NaN,050013
4,1000181_2,Endovasc DTA WO LT S&I,1,0320,75957,,NaN,Gross Charge,4288.00,gross,,,,75957,75957,NaN,NaN,050013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1205188,P9016,"RED BLOOD CELLS, LEUKOCYTES REDUCED, EACH UNIT",1,,P9016,,NaN,WESTERN_GROWERS,147.90,payer,ALL PLANS,,,NaN,NaN,NaN,NaN,054074
1205189,R079,"Chest pain, unspecified",1,,R079,,NaN,WESTERN_GROWERS,4183.40,payer,ALL PLANS,,,NaN,NaN,R079,NaN,054074
1205190,S0119,"ONDANSETRON, ORAL, 4 MG (FOR CIRCUMSTANCES FAL...",1,,S0119,,NaN,WESTERN_GROWERS,160.08,payer,ALL PLANS,,,NaN,NaN,NaN,NaN,054074
1205191,U0003,INFECTIOUS AGENT DETECTION BY NUCLEIC ACID (DN...,1,,U0003,,NaN,WESTERN_GROWERS,61.48,payer,ALL PLANS,,,NaN,NaN,NaN,NaN,054074


In [44]:
df2.drop_duplicates(subset=['hospital_id'], inplace=True)

In [45]:
df2

,local_code,description,setting,rev_code,code,ndc,drug_hcpcs_multiplier,payer,rate,payer_category,plan,modifiers,ms_drg,temp_cpt,hcpcs_cpt,icd,additional_payer_notes,hospital_id
0,1000006_1,Culture Wound wo Gram Stain,1,0306,87070,,<NA>,Gross Charge,490.0,gross,,,,87070,87070,NaN,NaN,054074
